In [ ]:
!pip install tensorflow matplotlib numpy


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense


In [ ]:
# Vocabulary size (top words only)
max_features = 10000
maxlen = 200  # each review cut/padded to 200 words

print("Loading data...")
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

print(len(x_train), "train sequences")
print(len(x_test), "test sequences")

# Pad sequences to fixed length
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)


In [ ]:
# Vocabulary size (top words only)
max_features = 10000
maxlen = 200  # each review cut/padded to 200 words

print("Loading data...")
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

print(len(x_train), "train sequences")
print(len(x_test), "test sequences")

# Pad sequences to fixed length
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)


In [ ]:
model = Sequential()
model.add(Embedding(max_features, 128))        # word embeddings
model.add(SimpleRNN(128, dropout=0.2))         # Simple RNN
model.add(Dense(1, activation='sigmoid'))      # output layer (binary)

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())


In [ ]:
history = model.fit(x_train, y_train,
                    batch_size=32,
                    epochs=5,
                    validation_data=(x_test, y_test))


In [ ]:
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()


In [ ]:
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence

# Load word index from IMDB
word_index = imdb.get_word_index()
reverse_word_index = {value: key for key, value in word_index.items()}

# Function to encode user review into integer sequence
def encode_review(text, word_index, maxlen=200):
    words = text.lower().split()
    encoded = []
    for word in words:
        index = word_index.get(word)
        if index is not None and index < 10000:  # keep only top words
            encoded.append(index + 3)  # +3 offset for reserved indices
        else:
            encoded.append(2)  # 2 = "unknown"
    return sequence.pad_sequences([encoded], maxlen=maxlen)

# ---- Take input from user ----
user_review = input("Enter your movie review: ")

# Preprocess the input review
processed_review = encode_review(user_review, word_index, maxlen=200)

# Predict sentiment
prediction = model.predict(processed_review)[0][0]

print("\nYour Review:", user_review)
print("Prediction:", "Positive 😀" if prediction > 0.5 else "Negative 😞")
print("Confidence Score:", prediction)


In [ ]:
# Train the model
history = model.fit(
    x_train,
    y_train,
    batch_size=64,
    epochs=5,
    validation_data=(x_test, y_test)
)

# Evaluate on test set
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f"Test Accuracy: {accuracy * 100:.2f}%")
